In [1]:
# Sample S&P program for one scrip

# STATUS: Completed
# Run-time: 1.3 mins

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7497, clientId=9) # rkv tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=9) # rkv IBG live

In [10]:
%%time
import numpy as np
import pandas as pd
from itertools import product
import datetime
import pickle
from math import sqrt, exp, log, erf

# #******   Error catch in list comprehension  ****
# #________________________________________________

# def catch(func, handle=lambda e : e, *args, **kwargs):
#     '''List comprehension error catcher'''
#     try:
#         return func(*args, **kwargs)
#     except Exception as e:
#         np.nan

# sd multiple for band
sigma = 2    # 2 sigma is about 95% probability
penalty = 1.1   # e.g. 1.1 is 20% above

minHzn = 20   # minimum option horizon
maxHzn = 90   # maximum option horizon

base = 0.01   # Upper or Lower base multiple for prices

blks = 50     # no of blocks to get data from

# market
exchange = 'SMART'
currency = 'USD'
commission = 0.7
tradedays = 252   # no of trading days in a year

# ... build the snp list

sym_chg_dict = {'BRK.B': 'BRK B', 'BRK/B': 'BRK B'} # Remap symbols in line with IBKR

snpurl = 'https://en.wikipedia.org/wiki/S%26P_100'
df_snp = pd.read_html(snpurl, header=0)[2]

df_snp.Symbol = df_snp.Symbol.map(sym_chg_dict).fillna(df_snp.Symbol)
df_snp['Type'] = 'Stock'

# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

# read from row no 11, dropna and reset index
df_cboe = pd.read_excel(dls, header=12, 
                        usecols=[0,2,3]).loc[11:, :]\
                        .dropna(axis=0)\
                        .reset_index(drop=True)

# remove column names white-spaces and remap to IBKR
df_cboe.columns = df_cboe.columns.str.replace(' ', '')
df_cboe.Ticker = df_cboe.Ticker.map(sym_chg_dict).fillna(df_cboe.Ticker)

# list the equities
equities = [e for e in list(df_snp.Symbol) if e in list(df_cboe.Ticker)]

# filter and list the etfs
df_etf = df_cboe[df_cboe.ProductType == 'ETF'].reset_index(drop=True)
etfs = list(df_etf.Ticker)

# list the indexes
indexes = 'OEX,XEO,XSP,DJX'.split(',')

# Build a list of contracts
ss = [Stock(symbol=s, currency=currency, exchange=exchange) for s in set(equities+etfs)]
ixs = [Index(symbol=s,currency=currency, exchange='CBOE') for s in set(indexes)]
cs = ss+ixs

qcs = ib.qualifyContracts(*cs) # qualified underlyings

#****           Single scrip check. To be DELETED in function          *****
#...........................................................................
contract = next(q for q in qcs if q.symbol=='CVX')  # one symbol logic check
#___________________________________________________________________________

#... Risk-free rate assumed
rate_url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield'
df_rate = pd.read_html(rate_url)[1]
df_rate.columns  = df_rate.iloc[0] # Set the first row as header
df_rate = df_rate.drop(0,0) # Drop the first row
rate = float(df_rate[-1:]['1 yr'].values[0])/100 # Get the last row's 1 yr value as float

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatality, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatality)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatality
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatality**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1
    
    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

#***   Error catching for list comprehension ***
#_______________________________________________
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan


#***   Get the ticker 456, which has the dividend  ***
#_____________________________________________________
def get_dividend_ticker(contract):
    '''Gets ticker of the contract
    Arg: (contract) as a qualified contract object with conId
    Returns: ticker'''
    
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream

    ticker = ib.ticker(contract)
    
    # Ensure the ticker is filled
    while ticker.close != ticker.close:
        ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False)
        ib.sleep(0.05)

    ib.cancelMktData(contract)
       
    return ticker

#****    Underlying data    ****
#_______________________________

#... Get the scrip
symbol = contract.symbol

#... Get volatility, hi52 and lo52
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)

stDev = np.std(a=[b.close for b in bars], ddof=0)

hi = max([b.high for b in bars])
lo = min([b.low for b in bars])

avg = np.mean([b.close for b in bars])

#... Get the lot, margin, undPrice and dividend rate
lot = 100

# margin of underlying
order = Order(action='SELL', totalQuantity=lot, orderType='MKT')
margin = float(ib.whatIfOrder(contract, order).initMarginChange)

ticker = get_dividend_ticker(contract)

# get the underlying price. If market it closed, take the close price, else the last price.
if np.isnan(ticker.last):
    undPrice = ticker.close
else:
    undPrice = ticker.last

# get the dividend
try:
    divrate = ticker.dividends.past12Months/undPrice
except (TypeError, AttributeError) as e:
    divrate = 0.0

#... Pickle the underlying dictionary
dictund = {'symbol':symbol, 'bars':bars, 'stDev': stDev, 
 'hi': hi, 'lo': lo, 'avg': avg, 'lot':lot, 'margin':margin, 
 'undPrice':undPrice, 'divrate':divrate, 'rate': rate, 'contract':contract}

with open('./zdata/_'+symbol+'.pkl', 'wb') as handle:
    pickle.dump(dictund, handle)
    

#****       Option data      ****
#________________________________

#... Get the option chain tickers, qualify them and get the prices
manychains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

chains = [c for c in manychains if c.exchange==exchange]

expiries =  set(sorted(exp for exp in chains[0].expirations 
                     if minHzn < (util.parseIBDatetime(exp)-datetime.datetime.now().date()).days < maxHzn))

cds = [ib.reqContractDetails(Option(symbol, e, exchange=exchange)) for e in expiries]

opts = [c.contract for cs in cds for c in cs]

#... Weed out unwanted SDs and make the option chain dataframe
# keep only the Ps and Cs outside the sigma band (95% probability)
options = [t for t in opts 
if ((t.strike < undPrice - stDev*sigma) & (t.right == 'P')) | 
((t.strike > undPrice + stDev*sigma) & (t.right == 'C'))]

# If any options are available in the range, move forward
if options != []:
    
    qo = [q for i in range(0, len(options), 40) for q in ib.qualifyContracts(*options[i:i+40])]

    df1 = util.df(qo)
    df1['opt_contract'] = qo

    tickers = [t for i in range(0, len(qo), 100) for t in ib.reqTickers(*qo[i:i + 100])]    
    ib.sleep(2)   # gives some time to fill the tickers    
    tickers = tickers

    df1['opt_ticker'] = tickers

    df1['dte'] = (df1.lastTradeDateOrContractMonth.apply(util.parseIBDatetime) - datetime.datetime.now().date()).dt.days

    # replace dte less than 0 with 0
    df1.loc[df1.dte<0, 'dte'] = 0

    df1['undPrice'] = undPrice
    df1['rate'] = rate
    df1['stDev'] = stDev
    df1['divrate'] = divrate

    # compute annualized volatility from running standard deviation for Black-Scholes
    df1['hv'] = [np.std(a=[b.close-b.open for b in bars[-dte:]], ddof=0)/undPrice*sqrt(tradedays) for dte in df1.dte]

    df2 = df1[['undPrice', 'strike', 'dte', 'rate', 'hv', 'divrate',
               'symbol', 'right', 'opt_contract', 'opt_ticker']]

    # get putprice, callprice and delta from bsm
    bsm = [catch(lambda: get_bsm(u, k, e, r, v, d)) for u, k, e, r, v, d in zip(df2.undPrice, df2.strike, df2.dte, df2.rate, df2.hv, df2.divrate)]

    # remove the Nones in the dictionary
    bsm = [{'bsmCall': np.nan, 'bsmPut': np.nan, 'bsmDelta': np.nan} if b is None else b for b in bsm]

    df_bsm = pd.DataFrame(list(bsm))

    df3 = pd.concat([df2, df_bsm],axis=1)

    df3['bsmPrice'] = np.where(df3.right == 'P', df3.bsmPut, df3.bsmCall)

    # Sort dataframe in ascending dte and descending strike
    df3 = df3.sort_values(by=['dte', 'strike'], ascending=[True, False]).reset_index(drop=True)

    df3['close'] = [catch(lambda: t.close) for t in df3.opt_ticker]
    df3['bid'] =  [catch(lambda: t.bid) for t in df3.opt_ticker]
    df3['ask'] = [catch(lambda: t.ask) for t in df3.opt_ticker]

    df3['lot'] = lot
    df3['margin'] = margin
    df3['hi'] = hi
    df3['lo'] = lo
    df3['avg'] = avg

    ibgreeks = [catch(lambda: (t.modelGreeks.impliedVol, t.modelGreeks.optPrice, t.modelGreeks.delta)) for t in df3.opt_ticker]
    ibgreeks = [(np.nan, np.nan, np.nan) if b is None else b for b in ibgreeks]  # for catch errors with None, replace with np.nan

    df_ib = pd.DataFrame(ibgreeks, columns=['ibiv', 'ibprice', 'ibdelta'])

    df3 = pd.concat([df3, df_ib], axis=1)

    df3['askbidavg'] = (df3.ask-df3.bid)/2

    df3['expPrice'] = round((df3[['askbidavg', 'close', 'bsmPrice']].max(axis=1)+(commission/lot))*penalty * 2, 1)/2

    df3['rom'] = (df3.expPrice*df3.lot)/df3.margin*tradedays/df3.dte
    df3['pop'] = np.where(df3.right == 'C', 1-df3.bsmDelta, df3.bsmDelta)

    cols = ['symbol', 'strike',  'dte',  'right', 'undPrice', 'pop', 'rom', 'expPrice', 'hv', 'bsmDelta',  'bsmPrice', 
            'ibiv', 'ibdelta', 'ibprice', 'close', 'bid', 'ask', 'lot', 'margin', 'hi', 'lo', 'avg', 'opt_ticker']
    df3[cols].to_pickle('./zdata/'+symbol+'.pkl')

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


Wall time: 44.1 s


,undPrice,strike,dte,rate,hv,divrate,symbol,right,opt_contract,opt_ticker,...,hi,lo,avg,ibiv,ibprice,ibdelta,askbidavg,expPrice,rom,pop
8,111.96,124.0,26,0.026,0.276291,0.040014,CVX,C,"Option(conId=350455349, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=350455349, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.35,0.309610,0.913368
9,111.96,100.0,26,0.026,0.276291,0.040014,CVX,P,"Option(conId=342779574, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=342779574, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.45,0.398070,0.940068
24,111.96,124.0,33,0.026,0.265304,0.040014,CVX,C,"Option(conId=348416024, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=348416024, symbol...",...,132.67,100.22,119.19241,0.179533,0.052688,0.024115,0.0,0.50,0.348479,0.895512
25,111.96,100.0,33,0.026,0.265304,0.040014,CVX,P,"Option(conId=348416077, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=348416077, symbol...",...,132.67,100.22,119.19241,0.271783,0.410929,-0.093146,0.0,0.55,0.383327,0.925090
26,111.96,99.5,33,0.026,0.265304,0.040014,CVX,P,"Option(conId=348416474, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=348416474, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.50,0.348479,0.933581
27,111.96,99.0,33,0.026,0.265304,0.040014,CVX,P,"Option(conId=348416489, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=348416489, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.50,0.348479,0.941344
28,111.96,98.5,33,0.026,0.265304,0.040014,CVX,P,"Option(conId=348416464, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=348416464, symbol...",...,132.67,100.22,119.19241,0.271783,0.271440,-0.065777,0.0,0.45,0.313631,0.948410
35,111.96,125.0,40,0.026,0.264472,0.040014,CVX,C,"Option(conId=349296909, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=349296909, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.55,0.316245,0.891051
36,111.96,124.0,40,0.026,0.264472,0.040014,CVX,C,"Option(conId=349296898, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=349296898, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.65,0.373744,0.872939
37,111.96,100.0,40,0.026,0.264472,0.040014,CVX,P,"Option(conId=349296989, symbol='CVX', lastTrad...","Ticker(contract=Option(conId=349296989, symbol...",...,132.67,100.22,119.19241,NaN,NaN,NaN,0.0,0.70,0.402493,0.906010
